In [15]:
LIGHT_IMPORT = True

# If the file is in the current folder:
%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity checks
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [16]:
"""
If any cells in Main Calc Embeddings are wrapped heavy cells those heavy steps 
(e.g., geocoding) will skip during this import.
"""

'\nIf any cells in Main Calc Embeddings are wrapped heavy cells those heavy steps \n(e.g., geocoding) will skip during this import.\n'

In [17]:
import numpy as np
import pandas as pd

In [18]:
def _ensure_text_sim(cands: pd.DataFrame) -> pd.DataFrame:
    c = cands.copy()
    if "text_sim" not in c.columns and "score" in c.columns:
        c["text_sim"] = c["score"]
    c["text_sim"] = c["text_sim"].fillna(0).clip(lower=0)
    return c

In [19]:
needed = ["top_k_semantic_neighbors_by_text","filter_by_orientation","filter_by_location",
          "filter_by_age","filter_by_diet","filter_by_drinks","filter_by_smokes","filter_by_drugs",
          "compute_component_scores","apply_weights","get_matches"]
print("Present:", {n: (n in globals()) for n in needed})

Present: {'top_k_semantic_neighbors_by_text': True, 'filter_by_orientation': True, 'filter_by_location': True, 'filter_by_age': True, 'filter_by_diet': True, 'filter_by_drinks': True, 'filter_by_smokes': True, 'filter_by_drugs': True, 'compute_component_scores': True, 'apply_weights': True, 'get_matches': True}


In [20]:
"""
verifies that the Phase-1 functions are actually loaded into this kernel.
prints a dict of name -> True/False. All should be True if Cell 0 succeeded.
"""

'\nverifies that the Phase-1 functions are actually loaded into this kernel.\nprints a dict of name -> True/False. All should be True if Cell 0 succeeded.\n'

In [21]:
# OkCupid-style compatibility scorer
"""
    Returns a DataFrame of top-k matches with 0–100% 'compat_pct' plus components.

    Hard filters: orientation, location (≤ max_km), age, and optionally diet/drinks/smokes/drugs if 
    hard_filters True.
    Soft scores:  text_sim, age_score (binary by your rule), lifestyle_score (avg), loc_score 
    (linear decay by distance/max_km).

"""
def okcupid_score(user_id, k=20, weights=None, pool_k=5000, max_km=50,allow_missing=True,
    drinks_strict=False, smokes_strict=False, drugs_strict=True, hard_filters=None):

    if weights is None:
        weights = {"text":0.70, "age":0.15, "life":0.10, "loc":0.05}
    if hard_filters is None:
        # default: similar to app OkCupid-ish — keep diet & drugs as gates, drinks/smokes as soft
        hard_filters = {"diet": True, "drinks": False, "smokes": False, "drugs": True}

    pool_k = min(int(pool_k), len(df) - 1)

    # 1) Hard-filtered candidate pool (preserves semantic order)
    cands = get_matches(user_id=user_id,k=pool_k,use_orientation=True,use_location=True, max_km=max_km,
        use_age=True, use_diet=hard_filters.get("diet", True), 
        use_drinks=hard_filters.get("drinks", False),  drinks_strict=drinks_strict,
        use_smokes=hard_filters.get("smokes", False),  smokes_strict=smokes_strict,
        use_drugs=hard_filters.get("drugs", True),     drugs_strict=drugs_strict,
        allow_missing=allow_missing
    ).copy()

    # 2) Component scores (adds text_sim/age_score/lifestyle_score/loc_score/distance_km)
    cands = compute_component_scores(user_id=user_id, cands=cands, max_km=max_km,
                                     drinks_strict=drinks_strict, smokes_strict=smokes_strict, 
                                     drugs_strict=drugs_strict,allow_missing=allow_missing)
    cands = _ensure_text_sim(cands)

    # 3) Weighted blend → 0–1, then convert to %
    ranked = apply_weights(cands, w_text=weights["text"], w_age=weights["age"], w_life=weights["life"],
                           w_loc=weights["loc"]).copy()
    ranked["compat_pct"] = (ranked["final_score"].clip(0, 1) * 100).round(1)

    # 4) Final shape
    want = ["user_id","compat_pct","text_sim","age_score","lifestyle_score","loc_score","distance_km",
            "location","age","sex","orientation"]
    for col in want:
        if col not in ranked.columns:
            ranked[col] = np.nan
    return ranked[want].head(k).reset_index(drop=True)


In [22]:
"""
def okcupid_score: 
    defines the wrapper you’ll call. k = how many results to return.
    pool_k = how many hard-filtered candidates to gather before re-ranking 

cands = get_matches: 
    builds the candidate pool by applying Phase-1 pipeline in “hard-filter” mode: mutual orientation, distance ≤ max_km, and age.
    optionally enforce lifestyle gates depending on hard_filters. 
    returns up to pool_k candidates (preserving the semantic order).

cands = compute_component_scores
    adds soft component columns to cands:
       text_sim (ensured in the next line),
       age_score (1.0 if mutual half+7 satisfied, else 0),
       lifestyle_score (avg across diet/drinks/smokes/drugs with strict/relaxed logic),
       loc_score (decays linearly with distance/max_km),
       distance_km 

ranked = computes final_score = (text*w1 + age*w2 + life*w3 + loc*w4)
         also normalizes weights inside apply_weights so any numbers sum to 1.
         maps the 0–1 blend to 0–100%, clipped and rounded to one decimal.
"""

'\ndef okcupid_score: \n    defines the wrapper you’ll call. k = how many results to return.\n    pool_k = how many hard-filtered candidates to gather before re-ranking \n\ncands = get_matches: \n    builds the candidate pool by applying Phase-1 pipeline in “hard-filter” mode: mutual orientation, distance ≤ max_km, and age.\n    optionally enforce lifestyle gates depending on hard_filters. \n    returns up to pool_k candidates (preserving the semantic order).\n\ncands = compute_component_scores\n    adds soft component columns to cands:\n       text_sim (ensured in the next line),\n       age_score (1.0 if mutual half+7 satisfied, else 0),\n       lifestyle_score (avg across diet/drinks/smokes/drugs with strict/relaxed logic),\n       loc_score (decays linearly with distance/max_km),\n       distance_km \n\nranked = computes final_score = (text*w1 + age*w2 + life*w3 + loc*w4)\n         also normalizes weights inside apply_weights so any numbers sum to 1.\n         maps the 0–1 blend to

In [23]:
okc = okcupid_score(
    user_id=1234,
    k=10,
    weights={"text":0.70, "age":0.15, "life":0.10, "loc":0.05},
    pool_k=5000,
    max_km=50,
    allow_missing=True,
    drinks_strict=False, smokes_strict=False, drugs_strict=True,
    hard_filters={"diet": True, "drinks": False, "smokes": False, "drugs": True}
)
okc

Start pool: 5000 (max_candidates=5000)
After orientation: 2023
Unfiltered: 2023 | After location filter (≤50 km): 1717
After location (≤50 km): 1717
After age rule: 1342
Diet| before: 1342 → after: 1152
After diet: 1152
Drugs  | before: 1152 → after: 1034 (strict=True)
After drugs (strict=True): 1034


,user_id,compat_pct,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,57256,68.1,0.544072,1.0,1.0,1.0,41.230915,"oakland, california",33,f,straight
1,53843,67.4,0.533671,1.0,1.0,1.0,41.230915,"oakland, california",47,f,straight
2,4015,67.2,0.531963,1.0,1.0,1.0,43.696864,"san francisco, california",38,f,straight
3,30039,67.2,0.531841,1.0,1.0,1.0,43.696864,"san francisco, california",24,f,straight
4,9558,66.6,0.522563,1.0,1.0,1.0,48.458096,"berkeley, california",46,f,straight
5,20404,66.3,0.518943,1.0,1.0,1.0,43.696864,"san francisco, california",43,f,straight
6,49650,66.2,0.517111,1.0,1.0,1.0,41.230915,"oakland, california",31,f,straight
7,33605,66.1,0.515604,1.0,1.0,1.0,30.936387,"castro valley, california",34,f,straight
8,7668,66.0,0.513741,1.0,1.0,1.0,41.230915,"oakland, california",43,f,bisexual
9,47133,65.9,0.513484,1.0,1.0,1.0,43.696864,"san francisco, california",29,f,straight


In [30]:
def add_reasons(df_in):
    df = df_in.copy()
    def _r(r):
        dist = "" if pd.isna(r.get("distance_km")) else f", ~{int(round(r.distance_km))} km"
        return f"text={r.text_sim:.2f}, age={int(r.age_score)}, life={r.lifestyle_score:.2f}, loc={r.loc_score:.2f}{dist}"
    df["reasons"] = df.apply(_r, axis=1)
    return df

add_reasons(okc).head(10)

,user_id,compat_pct,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,reasons
0,57256,68.1,0.544072,1.0,1.0,1.0,41.230915,"oakland, california",33,f,straight,"text=0.54, age=1, life=1.00, loc=1.00, ~41 km"
1,53843,67.4,0.533671,1.0,1.0,1.0,41.230915,"oakland, california",47,f,straight,"text=0.53, age=1, life=1.00, loc=1.00, ~41 km"
2,4015,67.2,0.531963,1.0,1.0,1.0,43.696864,"san francisco, california",38,f,straight,"text=0.53, age=1, life=1.00, loc=1.00, ~44 km"
3,30039,67.2,0.531841,1.0,1.0,1.0,43.696864,"san francisco, california",24,f,straight,"text=0.53, age=1, life=1.00, loc=1.00, ~44 km"
4,9558,66.6,0.522563,1.0,1.0,1.0,48.458096,"berkeley, california",46,f,straight,"text=0.52, age=1, life=1.00, loc=1.00, ~48 km"
5,20404,66.3,0.518943,1.0,1.0,1.0,43.696864,"san francisco, california",43,f,straight,"text=0.52, age=1, life=1.00, loc=1.00, ~44 km"
6,49650,66.2,0.517111,1.0,1.0,1.0,41.230915,"oakland, california",31,f,straight,"text=0.52, age=1, life=1.00, loc=1.00, ~41 km"
7,33605,66.1,0.515604,1.0,1.0,1.0,30.936387,"castro valley, california",34,f,straight,"text=0.52, age=1, life=1.00, loc=1.00, ~31 km"
8,7668,66.0,0.513741,1.0,1.0,1.0,41.230915,"oakland, california",43,f,bisexual,"text=0.51, age=1, life=1.00, loc=1.00, ~41 km"
9,47133,65.9,0.513484,1.0,1.0,1.0,43.696864,"san francisco, california",29,f,straight,"text=0.51, age=1, life=1.00, loc=1.00, ~44 km"


In [34]:
okcupid = okcupid_score(
    user_id=15,
    k=100,
    weights={"text":0.70, "age":0.15, "life":0.10, "loc":0.05},
    pool_k=500,
    max_km=45,
    allow_missing=True,
    drinks_strict=False, smokes_strict=True, drugs_strict=True,
    hard_filters={"diet": False, "drinks": False, "smokes": True, "drugs": True})
okcupid

Start pool: 5000 (max_candidates=5000)
After orientation: 2005
Unfiltered: 2005 | After location filter (≤45 km): 1941
After location (≤45 km): 1941
After age rule: 1478
Smokes | before: 1478 → after: 1277 (strict=True)
After smokes (strict=True): 1277
Drugs  | before: 1277 → after: 1277 (strict=True)
After drugs (strict=True): 1277


,user_id,compat_pct,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,28566,77.8,0.682167,1.0,1.0,1.0,0.000000,"san francisco, california",30,m,straight
1,49783,77.7,0.681057,1.0,1.0,1.0,13.300941,"oakland, california",42,m,straight
2,58244,77.1,0.672608,1.0,1.0,1.0,26.260308,"moraga, california",48,m,straight
3,20973,76.6,0.665855,1.0,1.0,1.0,32.302391,"san mateo, california",43,m,straight
4,33449,76.6,0.665802,1.0,1.0,1.0,0.000000,"san francisco, california",50,m,straight
...,...,...,...,...,...,...,...,...,...,...,...
95,19444,72.7,0.610466,1.0,1.0,1.0,21.350781,"pacifica, california",59,m,straight
96,20325,72.7,0.610337,1.0,1.0,1.0,0.000000,"san francisco, california",40,m,straight
97,41256,72.7,0.610004,1.0,1.0,1.0,0.000000,"san francisco, california",31,m,straight
98,45099,72.7,0.609931,1.0,1.0,1.0,16.407002,"berkeley, california",30,m,straight


In [36]:
import pandas as pd, json, os
from datetime import datetime

# make sure results folder exists
os.makedirs("results", exist_ok=True)

# add rank + metadata
okcupid = okcupid.reset_index(drop=True).copy()
okcupid["rank"] = range(1, len(okcupid)+1)
okcupid["app"] = "okcupid"

meta = {
    "app": "okcupid",
    "user_id": 15,
    "top_n": 100,
    "pool_k": 500,
    "max_km": 45,
    "weights": {"text":0.70, "age":0.15, "life":0.10, "loc":0.05},
    "filters": {"drinks_strict":False, "smokes_strict":True, "drugs_strict":True},
    "hard_filters": {"diet":False, "drinks":False, "smokes":True, "drugs":True},
    "timestamp": datetime.utcnow().isoformat() + "Z",
}

# choose a filename stem
stem = f"results/okcupid_user_id:{meta['user_id']}_k{meta['top_n']}_pool{meta['pool_k']}_km{meta['max_km']}"

# save to parquet + metadata json
okcupid.to_parquet(f"{stem}.parquet", index=False)
with open(f"{stem}.json","w") as f: json.dump(meta,f,indent=2)

print("Saved:", f"{stem}.parquet and {stem}.json")

Saved: results/okcupid_user_id:15_k100_pool500_km45.parquet and results/okcupid_user_id:15_k100_pool500_km45.json
